In [1]:
# Application of FL task
from MLModel import *
from FLModel import *
from utils import *

from torchvision import datasets, transforms
import torch
import numpy as np
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [2]:
def load_mnist(num_users):
    train = datasets.MNIST(root="~/data/", train=True, download=True, transform=transforms.ToTensor())
    train_data = train.data.float()
    train_label = train.targets

    mean = train_data.mean()
    std = train_data.std()
    train_data = (train_data - mean) / std

    test = datasets.MNIST(root="~/data/", train=False, download=True, transform=transforms.ToTensor())
    test_data = test.data.float()
    test_label = test.targets
    test_data = (test_data - mean) / std

    # split MNIST (training set) into non-iid data sets
    non_iid = []
    user_dict = mnist_noniid(train_label, num_users)
    for i in range(num_users):
        idx = user_dict[i]
        d = train_data[idx].flatten(1)
        targets = train_label[idx].float()
        non_iid.append((d, targets))
    non_iid.append((test_data.flatten(1).float(), test_label.float()))
    return non_iid

In [3]:
"""
1. load_data
2. generate clients (step 3)
3. generate aggregator
4. training
"""
client_num = 5
d = load_mnist(client_num)

In [4]:
"""
FL model parameters.
"""
lr = 0.1
fl_param = {
    'output_size': 10,
    'client_num': client_num,
    'model': MLP,
    'data': d,
    'lr': lr,
    'E': 100,
    'C': 1,
    'eps': 8.0,
    'delta': 1e-5,
    'q': 0.05,
    'clip': 4,
    'tot_T': 20,
    'batch_size': 64,
    'device': device
}
import warnings
warnings.filterwarnings("ignore")
fl_entity = FLServer(fl_param).to(device)

DP-SGD with sampling rate = 5% and noise_multiplier = 6.427124318662883 iterated over 40000 steps satisfies differential privacy with eps = 8 and delta = 1e-05.


In [5]:
acc = []
for t in range(fl_param['tot_T']):
    fl_entity.set_lr(lr/np.sqrt(t+1))
    acc += [fl_entity.global_update()]
    print("global epochs = {:d}, acc = {:.4f}".format(t+1, acc[-1]))

global epochs = 1, acc = 0.6658
global epochs = 2, acc = 0.7678
global epochs = 3, acc = 0.8175
global epochs = 4, acc = 0.8521
global epochs = 5, acc = 0.8594
global epochs = 6, acc = 0.8587
global epochs = 7, acc = 0.8657
global epochs = 8, acc = 0.8746
global epochs = 9, acc = 0.8760
global epochs = 10, acc = 0.8813
global epochs = 11, acc = 0.8833
global epochs = 12, acc = 0.8887
global epochs = 13, acc = 0.8915
global epochs = 14, acc = 0.8926
global epochs = 15, acc = 0.8956
global epochs = 16, acc = 0.8959
global epochs = 17, acc = 0.8965
global epochs = 18, acc = 0.8967
global epochs = 19, acc = 0.8974
global epochs = 20, acc = 0.8991
